<a href="https://colab.research.google.com/github/yahyachammami/Learn-Python-With-Yahia/blob/main/Extract_Information_From_Receipts_using_GEMINI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Yahia Chammami

In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 4.7 MB/s eta 0:00:00


In [2]:
import google.generativeai as genai

## SET API KEY

In [3]:
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## LIST OF MODELS

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
# Model Configuration
MODEL_CONFIG = {
  "temperature": 0.2,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

## Safety Settings of Model
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

## LOAD GEMINI MODEL WITH MODEL CONFIGURATIONS

In [6]:
model = genai.GenerativeModel(model_name = "gemini-pro-vision",
                              generation_config = MODEL_CONFIG,
                              safety_settings = safety_settings)

## DEFINE IMAGE FORMAT TO INPUT IN GEMINI

In [7]:
from pathlib import Path

def image_format(image_path):
    img = Path(image_path)

    if not img.exists():
        raise FileNotFoundError(f"Could not find image: {img}")

    image_parts = [
        {
            "mime_type": "image/png", ## Mime type are PNG - image/png. JPEG - image/jpeg. WEBP - image/webp
            "data": img.read_bytes()
        }
    ]
    return image_parts


## GEMINI MODEL OUTPUT

In [8]:
def gemini_output(image_path, system_prompt, user_prompt):

    image_info = image_format(image_path)
    input_prompt= [system_prompt, image_info[0], user_prompt]
    response = model.generate_content(input_prompt)
    return response.text

### EXTRACTING PART OF THE INFORMATION FROM INVOICE

In [9]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """

image_path = "/content/Invoice_Example.jpg"

user_prompt = "What is the total amount in the image?"

gemini_output(image_path, system_prompt, user_prompt)

' The total amount is 5650.00.'

## EXTRACTING WHOLE DATA IN JSON FROM INVOICE

In [10]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """
#system_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "
image_path = "/content/Invoice_Example.jpg"
user_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "

In [11]:

output = gemini_output(image_path, system_prompt, user_prompt)

In [12]:
from IPython.display import Markdown
Markdown(output)

 ```json
{
  "invoice_number": "000123",
  "invoice_date": "May 10, 2018",
  "payment_due": "May 30, 2018",
  "bill_to": {
    "name": "Bench Accounting",
    "address": "545 Robson St\nVancouver BC, V6B2B7 Canada",
    "email": "info@bench.co"
  },
  "ship_to": {
    "name": "Bench Accounting",
    "address": "545 Robson St\nVancouver BC, V6B2B7 Canada"
  },
  "order_date": "May 10, 2018",
  "est_delivery_date": "May 10, 2018",
  "method": "Ground",
  "trackable": "Yes",
  "note": "To be delivered to cargo bay #2",
  "items": [
    {
      "item": "Chairs",
      "description": "Comfortable but overpriced chairs",
      "quantity": 10,
      "unit_cost": 5000,
      "amount": 5000
    }
  ],
  "subtotal": 5000,
  "tax": 650,
  "total": 5650,
  "amount_due": 5650
}
```

In [13]:
user_promptt = "Convert Invoice data into dataframe with appropriate columns names as required for the data in image "
outputt = gemini_output(image_path, system_prompt, user_promptt)
from IPython.display import Markdown
Markdown(outputt)


 | Invoice Number | Invoice Date | Payment Due | Order Date | Shipping Method | Tracking Number | Item | Quantity | Unit Cost | Amount |
|---|---|---|---|---|---|---|---|---|---|
| 000123 | 05/10/2018 | 05/30/2018 | 05/10/2018 | Ground | Yes | 10 Chairs | 10 | $500.00 | $5,000.00 |
|  |  |  |  |  |  | Comfortable but overpriced chairs |  |  |  |
|  |  |  |  |  |  | Subtotal |  |  | $5,000.00 |
|  |  |  |  |  |  | Tax (13.0%) |  |  | $650.00 |
|  |  |  |  |  |  | Total |  |  | $5,650.00 |